<a href="https://colab.research.google.com/github/rehmanaly0051/Pytorch-IN-DeepLearning/blob/main/01_pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PYTORCH workflow
## https://www.learnpytorch.io/01_pytorch_workflow/

In [3]:
covering = {1: 'data(prepare and load)',
            2: 'build model',
            3: 'fitting the model to data(training)',
            4: 'make predictions and evaluating the model',
            5: 'savig and reloading the model',
            6: 'putting it all together'}
covering

{1: 'data(prepare and load)',
 2: 'build model',
 3: 'fitting the model to data(training)',
 4: 'make predictions and evaluating the model',
 5: 'savig and reloading the model',
 6: 'putting it all together'}

In [5]:
import torch
from torch import nn
import matplotlib.pyplot as plt

False